# ABD2

Wij gaan tijdens deze opdracht een food-collector simulatie bouwen.
In deze simulatie is het doel van de agents om zo veel mogelijk groen eten op te eten, terwijl ze rood eten vermijden.
We gaan deze simulatie grid-based maken. 

We maken gebruik van de gymnasium API: https://gymnasium.farama.org/

Ter visualisatie gebruiken we pygame.



### Simulation properties:
We beginnen in principe met een grid van 128x128, maar dit kan uitgebreid worden. 
We zullen de implementatie zoveel mogelijk scalable en aanpasbaar implementeren, door hyperparameters aan te maken voor alle belangrijke properties.

- Number of agents
- Grid size (default 128x128)
- Good food to total square ratio
- Good food spawning pattern (maybe)
- Bad food to total square ratio
- Bad food spawning pattern (maybe)
- Episode duration

### Agent properties:
Actions: up, down, left, right, wait

Perception: full information (knows coordinates of other agents, good food, and bad food)

Agent rules: not yet specified





In [1]:
import numpy as np
import pygame
import gymnasium as gym
from gymnasium import spaces
import random
import time

pygame 2.1.3 (SDL 2.0.22, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class agent():
    def __init__(self):
        self.location = [None, None]
        self.points = 0
        self._action_to_direction = {
            0: np.array([0, 0]),
            1: np.array([1, 0]),
            2: np.array([0, 1]),
            3: np.array([-1, 0]),
            4: np.array([0, -1])
        }

    def move(self, direction, grid_size):
        new_location = self.location + direction
        self.location = list(new_location)
        
    def pick_action(self,obs, grid_size):
        
        viable_actions = list(self._action_to_direction.keys())
        
        other_agent_locations = [x for x in obs['agent_locs'] if x != self.location]
        
        # Rule 1: agents can't move to a space occupied by another agent
        viable_actions = [v for v in viable_actions if list(self.location +  self._action_to_direction[v]) not in other_agent_locations]
        # Rule 2: agents can't move out of bounds
        viable_actions = [v for v in viable_actions
                          if list(self.location +  self._action_to_direction[v])[0] >= 0
                          and list(self.location +  self._action_to_direction[v])[1] >= 0
                          and list(self.location +  self._action_to_direction[v])[0] < grid_size
                          and list(self.location +  self._action_to_direction[v])[1] < grid_size]
        

        return random.choice(viable_actions)
        


In [3]:
class foodCollectorEnv(gym.Env):
    metadata = {"render_modes": ["human", None], "render_fps": 4}

    def __init__(self, render_mode=None, grid_size=128, game_duration=200, agents=1, good_food_ratio=0.1,
                 bad_food_ratio=0.1):
        self.grid_size = grid_size  # The size of the square grid
        self.window_size = 1028  # The size of the PyGame window
        self.game_duration = game_duration * agents
        self.current_step = 0
        self.agents = [agent() for _ in range(agents)]
        self.good_food_ratio = good_food_ratio
        self.bad_food_ratio = bad_food_ratio
        self.good_food_points = 5
        self.bad_food_points = -1
        self.render_mode = render_mode
        self.window = None


        self.observation_space = spaces.Dict({
            'agent_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
            'good_food_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
            'bad_food_locs': spaces.Sequence(spaces.MultiDiscrete([self.grid_size - 1, self.grid_size - 1])),
        })

        self.action_space = spaces.Sequence(spaces.Tuple((spaces.MultiDiscrete(len(self.agents)) ,spaces.MultiDiscrete(5) )))

        self._action_to_direction = {
            0: np.array([0, 0]),
            1: np.array([1, 0]),
            2: np.array([0, 1]),
            3: np.array([-1, 0]),
            4: np.array([0, -1])
        }

        self.good_food_locations = []
        self.bad_food_locations = []

    def _get_obs(self):
        return {
            "agent_locs": [agent.location for agent in self.agents],
            "good_food_locs": self.good_food_locs,
            "bad_food_locs": self.bad_food_locs
        }

    def reset(self, seed=None, options=None):

        super().reset(seed=seed)

        self.agent_locs = []
        self.good_food_locs = []
        self.bad_food_locs = []
            
        
        # place agents
        while True:
            if len(self.agent_locs) >= len(self.agents):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs:
                self.agent_locs.append(loc)
                

        # place good food
        while True:
            if len(self.good_food_locs) >= round(self.good_food_ratio * self.grid_size * self.grid_size):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs and loc not in self.good_food_locs and loc not in self.bad_food_locs:
                self.good_food_locs.append(loc)
        
        #place bad food
        while True:
            if len(self.bad_food_locs) >= round(self.bad_food_ratio * self.grid_size * self.grid_size):
                break
            loc = [random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)]
            if loc not in self.agent_locs and loc not in self.good_food_locs and loc not in self.bad_food_locs:
                self.bad_food_locs.append(loc)

        for i, agent in enumerate(self.agents):
            agent.location = self.agent_locs[i]

        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        info = {"agent_points": [agent.points for agent in self.agents]}

        return observation, info

    def step(self, action):

        direction = self._action_to_direction[action[1]]
        
        self.agents[action[0]].move(direction, self.grid_size)

        if self.agents[action[0]].location in self.good_food_locs:
            self.good_food_locs.remove(self.agents[action[0]].location)
            self.agents[action[0]].points += self.good_food_points

        if self.agents[action[0]].location in self.bad_food_locs:
            self.bad_food_locs.remove(self.agents[action[0]].location)
            self.agents[action[0]].points += self.bad_food_points

        terminated = self.current_step >= self.game_duration
        reward = 1 if terminated else 0  # 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()
        info = {"agent_points": [agent.points for agent in self.agents]}

        self.current_step += 1
        time.sleep(0)

        return observation, reward, terminated, info

    def _is_empty(self, loc):

        for agent in self.agents:
            if sorted(agent.location) == sorted(loc):
                return False

        return True

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        pygame.font.init()
        font = pygame.font.Font(None, 24)
        scoreboard_width = 0.2

        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size + self.window_size * scoreboard_width, self.window_size)
            )
     
        canvas = pygame.Surface((self.window_size + self.window_size * scoreboard_width, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
                self.window_size / self.grid_size
        )
        

        for i, agent in enumerate(self.agents):
            pygame.draw.rect(
                canvas,
                color='gray',
                rect=[agent.location[0] * pix_square_size, (self.grid_size - 1 - agent.location[1]) * pix_square_size,
                      pix_square_size, pix_square_size],
                width=0
            )
            lion_image = pygame.image.load("lion.png").convert_alpha()
            lion_image = pygame.transform.scale(lion_image, (pix_square_size, pix_square_size))
            canvas.blit(lion_image, (agent.location[0] * pix_square_size, (self.grid_size - 1 - agent.location[1]) * pix_square_size))
            
            score_text = font.render(f"{i}", True, "white")
            canvas.blit(score_text, (agent.location[0] * pix_square_size, (self.grid_size - 1 - agent.location[1]) * pix_square_size))
            
        

        for gfl in self.good_food_locs:
            pygame.draw.rect(
                canvas,
                color='green',
                rect=[gfl[0] * pix_square_size, (self.grid_size - 1 - gfl[1]) * pix_square_size, pix_square_size,
                      pix_square_size],
                width=0
            )

        for bfl in self.bad_food_locs:
            pygame.draw.rect(
                canvas,
                color='red',
                rect=[bfl[0] * pix_square_size, (self.grid_size - 1 - bfl[1]) * pix_square_size, pix_square_size,
                      pix_square_size],
                width=0
            )

        for x in range(self.grid_size + 1):
            pygame.draw.line(
                canvas,
                color=0,
                start_pos=(0, pix_square_size * x),
                end_pos=(self.window_size, pix_square_size * x),
                width=1,
            )
            pygame.draw.line(
                canvas,
                color=0,
                start_pos=(pix_square_size * x, 0),
                end_pos=(pix_square_size * x, self.window_size),
                width=1,
            )
            
        for i, agent in enumerate(self.agents):
            
            score_text = font.render(f'Agent {i} | {agent.points}', True, "black")
            canvas.blit(score_text, (self.window_size + 10 , 10 + (i*50)))

        if self.render_mode == "human":
           
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            #self.clock.tick(self.metadata["render_fps"])
        else:
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )


In [ ]:
env = foodCollectorEnv(render_mode="human", grid_size=32, game_duration=500, agents=10, good_food_ratio=0.02,
                       bad_food_ratio=0.2)

In [ ]:
env.reset()